In [86]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

**Import/Clean/Prep DataFrame for Modeling**

In [87]:
file_path = r"C:\Users\ianro\Downloads\MLB-player-final-dataset.csv"
modelage_df = pd.read_csv(file_path)

modelage_df.head()

,Unnamed: 0,IDfg,Season,Name,Age,G,AB,PA,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,LD%,GB%,FB%,HR/FB,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,CStr%,CSW%,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,F-Strike%,SwStr%,Zone%,Clutch,UBR,WAR,age_bracket
0,0,18401,2023,Ronald Acuna Jr.,25,159,643,735,0.109,0.114,0.95,0.416,0.596,1.012,0.258,0.338,0.201,0.495,0.304,0.240,0.397,0.356,0.247,0.101,0.438,0.461,94.7,7.4,86,0.153,121.2,309,0.550,0.137,0.214,0.268,0.728,0.738,0.878,0.589,0.078,0.422,1.16,2.2,8.4,2
1,1,13611,2023,Mookie Betts,30,152,584,693,0.139,0.154,0.90,0.408,0.579,0.987,0.272,0.316,0.237,0.278,0.485,0.167,0.463,0.355,0.183,0.100,0.490,0.411,92.4,20.6,60,0.125,110.1,232,0.481,0.205,0.262,0.200,0.630,0.643,0.939,0.595,0.058,0.421,-0.72,2.3,8.3,3
2,2,5361,2023,Freddie Freeman,33,161,637,730,0.099,0.166,0.60,0.410,0.567,0.976,0.235,0.370,0.276,0.353,0.370,0.150,0.365,0.357,0.278,0.106,0.526,0.369,90.0,15.2,58,0.111,110.6,218,0.418,0.114,0.208,0.301,0.792,0.688,0.880,0.614,0.094,0.426,-1.50,1.4,7.9,3
3,3,14344,2023,Matt Olson,29,162,608,720,0.144,0.232,0.62,0.389,0.604,0.993,0.321,0.302,0.178,0.387,0.436,0.278,0.382,0.373,0.245,0.112,0.461,0.427,93.7,16.1,73,0.164,118.6,245,0.551,0.131,0.249,0.295,0.735,0.695,0.777,0.556,0.119,0.395,-0.60,2.3,6.7,3
4,4,19755,2023,Shohei Ohtani,28,135,497,599,0.152,0.239,0.64,0.412,0.654,1.066,0.350,0.342,0.182,0.423,0.395,0.312,0.389,0.361,0.249,0.098,0.473,0.429,94.4,13.2,70,0.193,118.6,193,0.533,0.125,0.265,0.333,0.707,0.565,0.814,0.571,0.139,0.393,-2.08,1.7,6.6,2


In [88]:
columns_to_drop = ["Unnamed: 0", "IDfg", "Name", "Age", "EV", "LA", "Barrel%", "HardHit%", "maxEV", "UBR"]

modelage_df.drop(columns=columns_to_drop, inplace=True)

pd.set_option('display.max_columns', None)  
modelage_df.head()

,Season,G,AB,PA,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,LD%,GB%,FB%,HR/FB,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,Barrels,HardHit,CStr%,CSW%,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,F-Strike%,SwStr%,Zone%,Clutch,WAR,age_bracket
0,2023,159,643,735,0.109,0.114,0.95,0.416,0.596,1.012,0.258,0.338,0.201,0.495,0.304,0.240,0.397,0.356,0.247,0.101,0.438,0.461,86,309,0.137,0.214,0.268,0.728,0.738,0.878,0.589,0.078,0.422,1.16,8.4,2
1,2023,152,584,693,0.139,0.154,0.90,0.408,0.579,0.987,0.272,0.316,0.237,0.278,0.485,0.167,0.463,0.355,0.183,0.100,0.490,0.411,60,232,0.205,0.262,0.200,0.630,0.643,0.939,0.595,0.058,0.421,-0.72,8.3,3
2,2023,161,637,730,0.099,0.166,0.60,0.410,0.567,0.976,0.235,0.370,0.276,0.353,0.370,0.150,0.365,0.357,0.278,0.106,0.526,0.369,58,218,0.114,0.208,0.301,0.792,0.688,0.880,0.614,0.094,0.426,-1.50,7.9,3
3,2023,162,608,720,0.144,0.232,0.62,0.389,0.604,0.993,0.321,0.302,0.178,0.387,0.436,0.278,0.382,0.373,0.245,0.112,0.461,0.427,73,245,0.131,0.249,0.295,0.735,0.695,0.777,0.556,0.119,0.395,-0.60,6.7,3
4,2023,135,497,599,0.152,0.239,0.64,0.412,0.654,1.066,0.350,0.342,0.182,0.423,0.395,0.312,0.389,0.361,0.249,0.098,0.473,0.429,70,193,0.125,0.265,0.333,0.707,0.565,0.814,0.571,0.139,0.393,-2.08,6.6,2


In [89]:
modelage_df = modelage_df.dropna(subset=['O-Contact%'])

In [90]:
y=modelage_df["age_bracket"]

x=modelage_df.drop(columns=["age_bracket"])

In [91]:
y.shape

(7566,)

In [92]:
print(x)

      Season    G   AB   PA    BB%     K%  BB/K    OBP    SLG    OPS    ISO  \
0       2023  159  643  735  0.109  0.114  0.95  0.416  0.596  1.012  0.258   
1       2023  152  584  693  0.139  0.154  0.90  0.408  0.579  0.987  0.272   
2       2023  161  637  730  0.099  0.166  0.60  0.410  0.567  0.976  0.235   
3       2023  162  608  720  0.144  0.232  0.62  0.389  0.604  0.993  0.321   
4       2023  135  497  599  0.152  0.239  0.64  0.412  0.654  1.066  0.350   
...      ...  ...  ...  ...    ...    ...   ...    ...    ...    ...    ...   
7562    2013  107  365  399  0.068  0.095  0.71  0.288  0.310  0.597  0.074   
7563    2013  117  423  451  0.044  0.091  0.49  0.283  0.317  0.600  0.064   
7564    2013  137  391  409  0.034  0.174  0.20  0.240  0.355  0.595  0.143   
7565    2013   95  247  262  0.038  0.145  0.26  0.276  0.312  0.588  0.073   
7566    2013   67  223  243  0.062  0.218  0.28  0.263  0.265  0.528  0.054   

      BABIP    LD%    GB%    FB%  HR/FB  Pull%  Cen

In [93]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

**Check for NaN's** 

In [94]:
nan_count = modelage_df.isnull().sum()
print(nan_count)

Season         0
G              0
AB             0
PA             0
BB%            0
K%             0
BB/K           0
OBP            0
SLG            0
OPS            0
ISO            0
BABIP          0
LD%            0
GB%            0
FB%            0
HR/FB          0
Pull%          0
Cent%          0
Oppo%          0
Soft%          0
Med%           0
Hard%          0
Barrels        0
HardHit        0
CStr%          0
CSW%           0
O-Swing%       0
Z-Swing%       0
O-Contact%     0
Z-Contact%     0
F-Strike%      0
SwStr%         0
Zone%          0
Clutch         0
WAR            0
age_bracket    0
dtype: int64


In [95]:
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (7566, 35)
Shape of y: (7566,)


In [96]:
nan_values_x = np.isnan(x)
nan_values_y = np.isnan(y)
print("NaN values in x:", np.sum(nan_values_x))
print("NaN values in y:", np.sum(nan_values_y))

NaN values in x: Season        0
G             0
AB            0
PA            0
BB%           0
K%            0
BB/K          0
OBP           0
SLG           0
OPS           0
ISO           0
BABIP         0
LD%           0
GB%           0
FB%           0
HR/FB         0
Pull%         0
Cent%         0
Oppo%         0
Soft%         0
Med%          0
Hard%         0
Barrels       0
HardHit       0
CStr%         0
CSW%          0
O-Swing%      0
Z-Swing%      0
O-Contact%    0
Z-Contact%    0
F-Strike%     0
SwStr%        0
Zone%         0
Clutch        0
WAR           0
dtype: int64
NaN values in y: 0


c:\Users\ianro\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


**Logistic Regression Model On Base DataFrame**

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=1)

classifier.fit(x_train, y_train)

accuracy = classifier.score(x_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.507399577167019


c:\Users\ianro\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**Model Evaluation**

In [18]:
predictions = classifier.predict(x_test)
confusion_matrix(y_test, predictions)

array([[  0, 187,   9,   0,   0],
       [  0, 871, 119,   0,   0],
       [  0, 504,  89,   0,   0],
       [  0,  88,  18,   0,   0],
       [  0,   3,   4,   0,   0]], dtype=int64)

In [21]:
target_names = ["19-23 (1)", "24-28 (2)", "29-34 (3)", "35-39 (4)", "40-44 (5)"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

   19-23 (1)       0.00      0.00      0.00       196
   24-28 (2)       0.53      0.88      0.66       990
   29-34 (3)       0.37      0.15      0.21       593
   35-39 (4)       0.00      0.00      0.00       106
   40-44 (5)       0.00      0.00      0.00         7

    accuracy                           0.51      1892
   macro avg       0.18      0.21      0.17      1892
weighted avg       0.39      0.51      0.41      1892



c:\Users\ianro\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ianro\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ianro\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Note: Model has highest accuracy between ages 24-28, but drops off dramatically before and after that age cohort

**Model Optimization Attempts**

In [23]:
#increase the max iterations

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=2500, random_state=1)

classifier.fit(x_train, y_train)

accuracy = classifier.score(x_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5253699788583509


c:\Users\ianro\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Note: Increasing max iterations improved the accuracy of the model marginally, but tops out at around 1000 iterations.

In [29]:
#consolidate columns 4 and 5

modelage_df.loc[modelage_df['age_bracket'].isin([4, 5]), 'age_bracket'] = 4

age_bracket_dist = modelage_df['age_bracket'].value_counts()
print(age_bracket_dist)

age_bracket
2    3770
3    2526
1     837
4     433
Name: count, dtype: int64


In [82]:
#New age distribution:

#19-23 = 1;
#24-28 = 2;
#29-34 = 3;
#35-44 = 4

In [83]:
#re-run logistic regression
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=2500, random_state=1)

classifier.fit(x_train, y_train)

accuracy = classifier.score(x_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5253699788583509


c:\Users\ianro\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Note: Consolidating brackets 4 and 5 made almost no difference in the accuracy of the model.

**Incorporating Standard Scaler and PCA**

In [84]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, y_test are your data splits

# Create a pipeline with StandardScaler, PCA, and LogisticRegression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=10)), 
    ('classifier', LogisticRegression())
])

# Fit the pipeline on the training data
pipeline.fit(x_train, y_train)

# Predict on the test data
predictions = pipeline.predict(x_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.5221987315010571


Note: Normalizing data and employing principal component analysis only marginally improved model accuracy. Reducing to 20, 15, 10, and 5 components yielded nearly identical results.

**Redoing Age Brackets**

In [39]:
file_path = r"C:\Users\ianro\Downloads\MLB-player-final-dataset.csv"
agereorg_df = pd.read_csv(file_path)

agereorg_df.head()

,Unnamed: 0,IDfg,Season,Name,Age,G,AB,PA,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,LD%,GB%,FB%,HR/FB,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,CStr%,CSW%,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,F-Strike%,SwStr%,Zone%,Clutch,UBR,WAR,age_bracket
0,0,18401,2023,Ronald Acuna Jr.,25,159,643,735,0.109,0.114,0.95,0.416,0.596,1.012,0.258,0.338,0.201,0.495,0.304,0.240,0.397,0.356,0.247,0.101,0.438,0.461,94.7,7.4,86,0.153,121.2,309,0.550,0.137,0.214,0.268,0.728,0.738,0.878,0.589,0.078,0.422,1.16,2.2,8.4,2
1,1,13611,2023,Mookie Betts,30,152,584,693,0.139,0.154,0.90,0.408,0.579,0.987,0.272,0.316,0.237,0.278,0.485,0.167,0.463,0.355,0.183,0.100,0.490,0.411,92.4,20.6,60,0.125,110.1,232,0.481,0.205,0.262,0.200,0.630,0.643,0.939,0.595,0.058,0.421,-0.72,2.3,8.3,3
2,2,5361,2023,Freddie Freeman,33,161,637,730,0.099,0.166,0.60,0.410,0.567,0.976,0.235,0.370,0.276,0.353,0.370,0.150,0.365,0.357,0.278,0.106,0.526,0.369,90.0,15.2,58,0.111,110.6,218,0.418,0.114,0.208,0.301,0.792,0.688,0.880,0.614,0.094,0.426,-1.50,1.4,7.9,3
3,3,14344,2023,Matt Olson,29,162,608,720,0.144,0.232,0.62,0.389,0.604,0.993,0.321,0.302,0.178,0.387,0.436,0.278,0.382,0.373,0.245,0.112,0.461,0.427,93.7,16.1,73,0.164,118.6,245,0.551,0.131,0.249,0.295,0.735,0.695,0.777,0.556,0.119,0.395,-0.60,2.3,6.7,3
4,4,19755,2023,Shohei Ohtani,28,135,497,599,0.152,0.239,0.64,0.412,0.654,1.066,0.350,0.342,0.182,0.423,0.395,0.312,0.389,0.361,0.249,0.098,0.473,0.429,94.4,13.2,70,0.193,118.6,193,0.533,0.125,0.265,0.333,0.707,0.565,0.814,0.571,0.139,0.393,-2.08,1.7,6.6,2


In [40]:
#check age distribution

age_counts = agereorg_df['Age'].value_counts()
print(age_counts)

Age
26    835
27    806
25    767
28    714
24    648
29    613
30    528
23    464
31    449
32    386
33    314
22    243
34    237
35    158
36    107
21    102
37     72
38     39
39     27
20     25
40     11
42      7
41      7
19      3
43      3
44      2
Name: count, dtype: int64


In [49]:
agereorg_df['age_bracket'] = pd.cut(agereorg_df['Age'], bins=[18, 23, 28, 34, 39, 44], labels=[1, 2, 3, 4, 5])

# Check the distribution by the new age brackets
age_bracket_dist_reorg = agereorg_df['age_bracket'].value_counts()
print(age_bracket_dist_reorg)

age_bracket
2    3770
3    2527
1     837
4     403
5      30
Name: count, dtype: int64


In [51]:
agereorg_df['age_bracket'] = pd.cut(agereorg_df['Age'], bins=[18, 24, 28, 34, 44], labels=[1, 2, 3, 4])

# Check the distribution by the new age brackets
age_bracket_dist_reorg = agereorg_df['age_bracket'].value_counts()
print(age_bracket_dist_reorg)

age_bracket
2    3122
3    2527
1    1485
4     433
Name: count, dtype: int64


In [52]:
agereorg_df['age_bracket'] = pd.cut(agereorg_df['Age'], bins=[18, 25, 28, 34, 44], labels=[1, 2, 3, 4])

# Check the distribution by the new age brackets
age_bracket_dist_reorg = agereorg_df['age_bracket'].value_counts()
print(age_bracket_dist_reorg)

age_bracket
3    2527
2    2355
1    2252
4     433
Name: count, dtype: int64


In [54]:
#clean new df, re-run model

columns_to_drop = ["Unnamed: 0", "IDfg", "Name", "Age", "EV", "LA", "Barrel%", "HardHit%", "maxEV", "UBR"]

agereorg_df.drop(columns=columns_to_drop, inplace=True)

pd.set_option('display.max_columns', None)  
agereorg_df.head()

,Season,G,AB,PA,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,LD%,GB%,FB%,HR/FB,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,Barrels,HardHit,CStr%,CSW%,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,F-Strike%,SwStr%,Zone%,Clutch,WAR,age_bracket
0,2023,159,643,735,0.109,0.114,0.95,0.416,0.596,1.012,0.258,0.338,0.201,0.495,0.304,0.240,0.397,0.356,0.247,0.101,0.438,0.461,86,309,0.137,0.214,0.268,0.728,0.738,0.878,0.589,0.078,0.422,1.16,8.4,1
1,2023,152,584,693,0.139,0.154,0.90,0.408,0.579,0.987,0.272,0.316,0.237,0.278,0.485,0.167,0.463,0.355,0.183,0.100,0.490,0.411,60,232,0.205,0.262,0.200,0.630,0.643,0.939,0.595,0.058,0.421,-0.72,8.3,3
2,2023,161,637,730,0.099,0.166,0.60,0.410,0.567,0.976,0.235,0.370,0.276,0.353,0.370,0.150,0.365,0.357,0.278,0.106,0.526,0.369,58,218,0.114,0.208,0.301,0.792,0.688,0.880,0.614,0.094,0.426,-1.50,7.9,3
3,2023,162,608,720,0.144,0.232,0.62,0.389,0.604,0.993,0.321,0.302,0.178,0.387,0.436,0.278,0.382,0.373,0.245,0.112,0.461,0.427,73,245,0.131,0.249,0.295,0.735,0.695,0.777,0.556,0.119,0.395,-0.60,6.7,3
4,2023,135,497,599,0.152,0.239,0.64,0.412,0.654,1.066,0.350,0.342,0.182,0.423,0.395,0.312,0.389,0.361,0.249,0.098,0.473,0.429,70,193,0.125,0.265,0.333,0.707,0.565,0.814,0.571,0.139,0.393,-2.08,6.6,2


In [55]:
agereorg_df = agereorg_df.dropna(subset=['O-Contact%'])

In [56]:
y=agereorg_df["age_bracket"]

x=agereorg_df.drop(columns=["age_bracket"])

In [59]:
y.shape

print(x)

      Season    G   AB   PA    BB%     K%  BB/K    OBP    SLG    OPS    ISO  \
0       2023  159  643  735  0.109  0.114  0.95  0.416  0.596  1.012  0.258   
1       2023  152  584  693  0.139  0.154  0.90  0.408  0.579  0.987  0.272   
2       2023  161  637  730  0.099  0.166  0.60  0.410  0.567  0.976  0.235   
3       2023  162  608  720  0.144  0.232  0.62  0.389  0.604  0.993  0.321   
4       2023  135  497  599  0.152  0.239  0.64  0.412  0.654  1.066  0.350   
...      ...  ...  ...  ...    ...    ...   ...    ...    ...    ...    ...   
7562    2013  107  365  399  0.068  0.095  0.71  0.288  0.310  0.597  0.074   
7563    2013  117  423  451  0.044  0.091  0.49  0.283  0.317  0.600  0.064   
7564    2013  137  391  409  0.034  0.174  0.20  0.240  0.355  0.595  0.143   
7565    2013   95  247  262  0.038  0.145  0.26  0.276  0.312  0.588  0.073   
7566    2013   67  223  243  0.062  0.218  0.28  0.263  0.265  0.528  0.054   

      BABIP    LD%    GB%    FB%  HR/FB  Pull%  Cen

In [60]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [62]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=2500, random_state=1)

classifier.fit(x_train, y_train)

accuracy = classifier.score(x_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.38636363636363635


c:\Users\ianro\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Note: Despite consolidating the number of age brackets to 4, and reorganizing the distribution so that 3 of the brackets were nearly equal in size, the accuracy of the model dropped significantly. 

**Normalize Data and Principal Component Analysis On Revised Age Bracket DataFrame**

In [63]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

agereorg_df_scaled = scaler.fit_transform(agereorg_df)

print(agereorg_df_scaled)

[[ 1.63789008  1.75966123  2.18463951 ...  2.03044735  4.83286632
  -1.25719152]
 [ 1.63789008  1.61820136  1.87624571 ... -1.11382538  4.76940169
   0.9279477 ]
 [ 1.63789008  1.80007834  2.15327743 ... -2.41836407  4.51554315
   0.9279477 ]
 ...
 [-1.543859    1.31507306  0.86743209 ...  1.49525199 -1.64052641
   0.9279477 ]
 [-1.543859    0.46631383  0.11474214 ... -0.22740807 -1.64052641
  -0.16462191]
 [-1.543859   -0.09952565 -0.01070619 ... -0.11033408 -1.83092032
  -1.25719152]]


In [65]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, y_test are your data splits

# Create a pipeline with StandardScaler, PCA, and LogisticRegression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=10)), 
    ('classifier', LogisticRegression())
])

# Fit the pipeline on the training data
pipeline.fit(x_train, y_train)

# Predict on the test data
predictions = pipeline.predict(x_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.36046511627906974


In [73]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, y_test are your data splits

# Create a pipeline with StandardScaler, PCA, and LogisticRegression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),  
    ('classifier', LogisticRegression())
])

# Fit the pipeline on the training data
pipeline.fit(x_train, y_train)

# Predict on the test data
predictions = pipeline.predict(x_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.35993657505285415


Note: Normalization and PCA actually slightly reduced the accuracy of the model. Original DataFrame with 4 unbalanced age brackets proved to be more accurate. 